In [3]:
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [4]:
from googleapiclient.discovery import build
youtube = build("youtube","v3",developerKey=API_KEY)

In [5]:
def get_mananera_ids(channel_id, max_results=5):
    response = youtube.search().list(
        q="#MañaneraDelPueblo",
        channelId=channel_id,
        part="snippet",
        order="date",
        maxResults=max_results,
        type="video"
    ).execute()

    video_ids = []
    for item in response["items"]:
        video_ids.append(item["id"]["videoId"])
    return video_ids

In [6]:
def get_comments(video_id, max_comments=1000):
    comments = []
    next_page_token = None
    while len(comments) < max_comments:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response.get("items", []):
            snippet = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "video_id": video_id,
                "comment_id": item["id"],
                "author_id": snippet.get("authorChannelId", {}).get("value", "NA"),
                "author_name": snippet.get("authorDisplayName", "NA"),
                "comment_text": snippet.get("textDisplay", "NA"),
                "published_at": snippet.get("publishedAt", "NA")
            })

        # mover fuera del for
        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break
    return comments

In [7]:
CHANNEL_ID = "UCvzHrtf9by1-UY67SfZse8w" 

ids = get_mananera_ids(CHANNEL_ID, max_results=5)

In [9]:
video_ids = get_mananera_ids(CHANNEL_ID,max_results=5)

all_comments = []

for vid in video_ids:
    comments = get_comments(vid)
    all_comments.extend(comments)

df = pd.DataFrame(all_comments)
df.to_csv("data/comments.csv")
